In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


# LOAD IMAGE

In [8]:
def read_file(filename):
    img = cv2.imread(filename)
#    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # you will use this in case you want to pring the image using plt show 
#    plt.imshow(img)
#    plt.show()
    cv2.imshow("Edges", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img

image_dir = os.path.join('../data')
image_filename = 'yahia-cartoon.jpg'
full_image_path = os.path.join(image_dir, image_filename)
img = cv2.imread(full_image_path)
cv2.imshow("Edges", img)
cv2.waitKey(0)

cv2.destroyAllWindows()



error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# Main charactarestics of the Cartoon image
## 1 - Clear Thick Edges
## 2 - It has clear homogeneus colors


#### 1) GET CLEAR EDGES

In [3]:

def get_edges(img, thickness, blurring):
    # Convert to gray scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # blurr the image in a way to smooth unnecessary edges
    gray_blur = cv2.medianBlur(gray, blurring)

    # Get the edges of the blurred img
    edges = cv2.adaptiveThreshold(
        gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, thickness, blurring)

    return edges




In [4]:
# This was by try and error. Blurring by 5 was enough (must be odd value)
edges = get_edges(img, 11, 9)
# Thickness 9 was good
cv2.imshow("Edges", edges)
cv2.waitKey(0)

cv2.destroyAllWindows()


#### 2) Clear Homogeneus Colors


In [5]:

# This may be done with segmentation in colors to select only 5-10 colors to apply on the image
def clear_colors(img, k):
    # Flatten the image
    data = np.float32(img).reshape((-1, 3))
    # Determine Criteria for K-Means Clustering
    criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)
    print(criteria)

    # Perform K-Means Clustering
    ret, label, center = cv2.kmeans(
        data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)

    result = center[label.flatten()]

    result = result.reshape(img.shape)

    return result


quantized_img = clear_colors(img, 16)

cv2.imshow("Homo", quantized_img)
cv2.waitKey(0)

cv2.destroyAllWindows()

(3, 20, 0.001)


##### Reduce some noise

In [6]:
blurred_img = cv2.bilateralFilter(quantized_img , d=7 , sigmaColor= 200, sigmaSpace=200)



cv2.imshow("Blurred", blurred_img)
cv2.waitKey(0)

cv2.destroyAllWindows()


#### 3) Cartoon by mixing them up


In [7]:


cartoon = cv2.bitwise_and(blurred_img, blurred_img, mask=edges)

cv2.imshow("Cartoon", cartoon)

cv2.waitKey(0)

cv2.destroyAllWindows()
